# Feature engineering

----------------------------

In [3]:
from google.colab import drive
drive.mount('/content/drive')
path='/content/drive/MyDrive/data_tesis_mds'

Mounted at /content/drive


In [4]:
import warnings
warnings.filterwarnings("ignore")

from pathlib import Path
import json
from io import StringIO

import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
from numpy import mean,sqrt,square
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from datetime import timedelta
from tqdm.notebook import tqdm_notebook
from tqdm import tqdm
pd.set_option('display.max_columns', None)

In [5]:
#!pip install pacmap
#import pacmap
import plotly.io as pio
from IPython.display import Image
!pip install -U kaleido

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.4 MB/s eta 0:00:00


In [6]:
!pip install pandarallel
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=False, nb_workers=16)

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 2.3 MB/s eta 0:00:00
  Created wheel for pandarallel: filename=pandarallel-1.6.5-py3-none-any.whl size=16672 sha256=6ee88171efe30f9138faca951dc0d6f57f27ef2fc85a2ef192d6f69b01f38228
  Stored in directory: /root/.cache/pip/wheels/50/4f/1e/34e057bb868842209f1623f195b74fd7eda229308a7352d47f
Successfully built pandarallel
INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [17]:
tags=['granulometry', 'charge cell', 'speed', 'solid percentage', 'water','TPH', 'HH TPH', 'LL charge cell', 'HH charge cell',

#"loss of TPH","start loss of TPH"	,"end loss of TPH",		"start low load cell",	"end low load cell",		"start high load cell",	"end high load cell","high load cell","low load cell",
#"Optimizando tonelaje","Limitado por Peso",
"delta HH TPH","delta HH charge cell",
#'pyrite law', 'chalcopyrite law','chalcocite law', 'crusher index', 'ball work index','bornite law','covelin law',
'sag power index',

"Edad"
]

In [8]:
file_path = Path().joinpath(path,"data","processed data",'cleaned_08September2023.csv',)

with open(file_path, 'r') as file:
    content = file.read()

# parse consolidation to df
df_clean = pd.read_csv(StringIO(content),parse_dates=['Timestamp'], index_col='Timestamp')
df_clean.head()

,granulometry,charge cell,speed,solid percentage,water,TPH,HH TPH,LL charge cell,HH charge cell,loss of TPH,start loss of TPH,end loss of TPH,start low load cell,end low load cell,start high load cell,end high load cell,delta HH TPH,delta HH charge cell,high load cell,low load cell,pyrite law,chalcopyrite law,chalcocite law,crusher index,sag power index,ball work index,bornite law,covelin law,Edad
Timestamp,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2018-01-01 00:03:00,62.811786,858.914758,9.062208,65.085609,2089.536865,3894.792946,4350.0,820.0,900.0,1,0,0,0,0,0,0,455.207054,41.085242,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2018-01-01 00:04:00,62.749109,857.932221,9.062822,65.271563,2110.715057,3959.416847,4350.0,820.0,900.0,1,0,0,0,0,0,0,390.583153,42.067779,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2018-01-01 00:05:00,62.731017,857.681061,9.062958,65.213058,2122.028610,3972.132198,4350.0,820.0,900.0,1,0,0,0,0,0,0,377.867802,42.318939,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2018-01-01 00:06:00,62.681308,860.322055,9.061771,65.283561,2141.409208,4023.434325,4350.0,820.0,900.0,1,0,0,0,0,0,0,326.565675,39.677945,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2018-01-01 00:07:00,62.677879,863.085639,9.062551,65.206578,2155.794501,4036.253188,4350.0,820.0,900.0,1,0,0,0,0,0,0,313.746812,36.914361,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [18]:
def feature_engineering(df):

    targets = [""]
    df_vars = df.copy()
    column_inicial=df_vars.columns

    for column in tqdm_notebook(tags,desc=f"Feature Engineering tags rolling"):

        if (column not in targets):

            ## 10 min
            #df_vars[f"skew_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_10"]=df_vars[column].rolling(window=10).parallel_apply(lambda x: np.nanvar(x))

            # 5 min
            df_vars[f"mean_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_5"]=df_vars[column].rolling(window=5).parallel_apply(lambda x: np.nanvar(x))

            # 3 min
            #df_vars[f"skew_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:skew(x))
            #df_vars[f"kurt_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:kurtosis(x))
            df_vars[f"mean_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:np.nanmean(list(x)))
            df_vars[f"max_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:np.nanmax(list(x)))
            #df_vars[f"sum_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:np.nansum(list(x)))
            df_vars[f"min_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x:np.nanmin(list(x)))
            df_vars[f"rms_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x: sqrt(mean(square(list(x)))))
            df_vars[f"var_{column}_3"]=df_vars[column].rolling(window=3).parallel_apply(lambda x: np.nanvar(x))


    for column in tqdm_notebook(tags,desc=f"Feature Engineering tags shift"):

        if (column not in targets):

            # lags
            df_vars[f"{column}_(t-1)"] = df_vars[column].shift(1)
            df_vars[f'{column}_(t-2)'] = df_vars[column].shift(2)
            df_vars[f'{column}_(t-3)'] = df_vars[column].shift(3)
            df_vars[f'{column}_(t-4)'] = df_vars[column].shift(4)
            df_vars[f'{column}_(t-5)'] = df_vars[column].shift(5)


    return df_vars


In [ ]:
df_fe=feature_engineering(df_clean)

Feature Engineering tags rolling:   0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
file_path = Path().joinpath(path,"data","processed data","feature_engineering_"+pd.to_datetime('today').strftime('%d%B%Y')+'.csv',)
df_fe.to_csv(file_path,index=True)
print(file_path)

In [ ]:
file_path = Path().joinpath(path,"data","processed data","feature_engineering_"+pd.to_datetime('today').strftime('%d%B%Y')+'.csv',)

with open(file_path, 'r') as file:
    content = file.read()

# parse consolidation to df
df_fe = pd.read_csv(StringIO(content),parse_dates=['Timestamp'], index_col='Timestamp')
df_fe.head()